In [1]:
import numpy as np

In [2]:
from PIL import Image

img = Image.open('src/blue.jpg')
pixels = list(img.getdata())
width, height = img.size

# Преобразуем в матрицу
matrix = [[pixels[y * width + x] for x in range(width)] for y in range(height)]

In [3]:
def rgb_to_grayscale(matrix):
    height = len(matrix)
    width = len(matrix[0])
    gray = [[0 for _ in range(width)] for _ in range(height)]

    for y in range(height):
        for x in range(width):
            r, g, b = matrix[y][x]
            gray[y][x] = int(0.299 * r + 0.587 * g + 0.114 * b)

    return gray

In [4]:
def matrix_to_image(matrix):
    height = len(matrix)
    width = len(matrix[0])

    # Преобразуем матрицу в плоский список пикселей
    flat_pixels = [pixel for row in matrix for pixel in row]

    # Создаём новое изображение
    img = Image.new('RGB', (width, height))
    img.putdata(flat_pixels)
    
    return img

In [5]:
gray_matrix = rgb_to_grayscale(matrix)

In [6]:
def is_background(intensity):
        background_threshold = 100
        return intensity > background_threshold

def find_most_different_object(gray_matrix):
    
    height = len(gray_matrix)
    width = len(gray_matrix[0])

    visited = [[False for _ in range(width)] for _ in range(height)]

    objects = []

    for y in range(height):
        for x in range(width):
            if visited[y][x] or is_background(gray_matrix[y][x]):
                continue

            object_pixels = []
            stack = [(y, x)]
            visited[y][x] = True

            while stack:
                curr_y, curr_x = stack.pop()
                object_pixels.append((curr_y, curr_x))

                for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    ny, nx = curr_y + dy, curr_x + dx
                    if (0 <= ny < height and 0 <= nx < width and 
                        not visited[ny][nx] and not is_background(gray_matrix[ny][nx])):
                        visited[ny][nx] = True
                        stack.append((ny, nx))

            y_coords = [p[0] for p in object_pixels]
            x_coords = [p[1] for p in object_pixels]
            y_min, y_max = min(y_coords), max(y_coords)
            x_min, x_max = min(x_coords), max(x_coords)

            total_intensity = sum(gray_matrix[py][px] for py, px in object_pixels)
            avg_intensity = int(total_intensity / len(object_pixels))

            objects.append({
                'bbox': (x_min, y_min, x_max, y_max),
                'pixels': object_pixels,
                'avg_intensity': avg_intensity,
                'area': len(object_pixels)
            })

    if not objects:
        return None
    
    most_prominent = max(objects, key=lambda obj: obj['area'])

    return most_prominent

In [7]:
result = find_most_different_object(gray_matrix)

print(f"Границы объекта: {result['bbox']}")
print(f"Площадь: {result['area']} пикселей")
print(f"Средняя яркость: {result['avg_intensity']}")

Границы объекта: (141, 31, 463, 431)
Площадь: 91426 пикселей
Средняя яркость: 61


In [8]:
def create_object_only_image(original_matrix, object_info):
    
    height = len(original_matrix)
    width = len(original_matrix[0])
    
    # Создаем новую матрицу, заполненную белым цветом
    result_matrix = [[(255, 255, 255) for _ in range(width)] for _ in range(height)]
    
    # Копируем пиксели объекта из оригинальной матрицы
    for y, x in object_info['pixels']:
        result_matrix[y][x] = original_matrix[y][x]
    
    return result_matrix

In [9]:
result_matrix = create_object_only_image(matrix, result)

In [10]:
resul_img = matrix_to_image(result_matrix)

In [11]:
resul_img.show()